In [ ]:
from mimic.utilities import *

from mimic.model_infer.infer_CRM_bayes import *
from mimic.model_infer import *
from mimic.model_simulate import *
from mimic.model_simulate.sim_CRM import *

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import arviz as az
import pymc as pm
import pytensor.tensor as at
import pickle
import cloudpickle


## Bayesian inference to infer the parameters of a Consumer Resource model

The Consumer Resource equation takes the form

$$
dN_i = \frac{N_i}{\tau} \cdot \left( c_{ij} \cdot (w \cdot R)_j - m_i \right)
$$

$$
dR_j = \frac{1}{r \cdot K_j} \cdot (K_j - R_j) \cdot R_j - (N_i \cdot c_{ij} \cdot R_j)
$$


where:
-   $N$ is the concentration of each species
-   $\tau$ is the species timescale
-   $m$ is the species mortality rate
-   $R$ is the concentration of each resource
-   $r$ is the resource timescale
-   $w$ is the quality of each resource
-   $K$ is the resource capacity
-   $c$ is each species' preference for each resource



In [ ]:
# read in pickled simulated parameters, tau, m, r, w, K and c
num_species = 2
with open("params-s2.pkl", "rb") as f:
    params = pickle.load(f)
tau = params["tau"]
m = params["m"]
r = params["r"]
w = params["w"]
K = params["K"]
c = params["c"]

# read in the data

num_timecourses = 1
data = pd.read_csv("data-s2-r1.csv")
times = data.iloc[:, 0].values

yobs = data.iloc[:, 1:6].values

X, F = linearize_time_course_16S(yobs, times)


# Define priors
prior_tau_mean = 1.0    # species timescale vector
prior_tau_sigma = 0.5

prior_m_mean = 1.0      # mortality rate vector
prior_m_sigma = 0.5

prior_r_mean = 1.0      # resource timescale vector
prior_r_sigma = 0.5

prior_w_mean = 1.0      # resource quality vector
prior_w_sigma = 0.5

prior_K_mean = 1.0      # resource carrying capacity vector
prior_K_sigma = 0.5

prior_c_mean = 1.0      # resource preference matrix
prior_c_sigma = 0.5

# Sampling conditions
draws = 500
tune = 500
chains = 4
cores = 4

inference = inferCRMbayes()

inference.set_parameters(X=X, F=F, 
                          prior_tau_mean=prior_tau_mean, prior_tau_sigma=prior_tau_sigma,
                          prior_m_mean=prior_m_mean, prior_m_sigma=prior_m_sigma,
                          prior_r_mean=prior_r_mean, prior_r_sigma=prior_r_sigma,
                          prior_w_mean=prior_w_mean, prior_w_sigma=prior_w_sigma,
                          prior_K_mean=prior_K_mean, prior_K_sigma=prior_K_sigma,
                          prior_c_mean=prior_c_mean, prior_c_sigma=prior_c_sigma,
                          draws=draws, tune=tune, chains=chains,cores=cores)

idata = inference.run_inference()


# To plot posterior distributions
#inference.plot_posterior(idata)

summary = az.summary(idata, var_names=["tau_hat", "m_hat", "r_hat", "w_hat", "K_hat", "c_hat", "sigma"])
print(summary[["mean", "sd", "r_hat"]])

# Save posterior samples to file
az.to_netcdf(idata, 'model_posterior.nc')


# compare fitted with simulated parameters
tau_h = np.median(idata.posterior["tau_hat"].values, axis=(0,1) ).reshape(-1)
m_h = np.median(idata.posterior["m_hat"].values, axis=(0,1) ).reshape(-1)
r_h = np.median(idata.posterior["r_hat"].values, axis=(0,1) ).reshape(-1)
w_h = np.median(idata.posterior["w_hat"].values, axis=(0,1) ).reshape(-1)
K_h = np.median(idata.posterior["K_hat"].values, axis=(0,1) ).reshape(-1)
c_h = np.median(idata.posterior["c_hat"].values, axis=(0,1) )

predictor = simCRM(num_species=num_species, num_resources=num_resources, tau=tau_h, m=m_h, r=r_h, w=w_h, K=K_h, c=c_h.T)
yobs_h, _, _, _, _ = predictor.simulate(times=times, init_species=yobs[0])


plot_fit_CRM(yobs, yobs_h, times)